In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Hotel Booking Cancellation Prediction - Colab Notebook
# Complete project in a single notebook

#@title ## Install and Import Required Libraries
!pip install xgboost pyyaml plotly -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (roc_curve, auc, precision_recall_curve,
                            confusion_matrix, classification_report,
                            roc_auc_score, accuracy_score, precision_score, recall_score, f1_score)
from sklearn.impute import SimpleImputer
import xgboost as xgb
import joblib
import warnings
import io
import requests
from google.colab import files
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Avoid line wrapping

print("All libraries imported successfully!")

All libraries imported successfully!


In [3]:
#@title Loading the Dataset

hotel_df = pd.read_csv('/content/drive/MyDrive/Hotel_Cancellation_Prediction/Dataset/hotel_booking_clean.csv')
hotel_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card,total_guests,total_stay,is_family,arrival_date,arrival_season,requests_per_night,high_lead_time,has_previous_cancellations
0,1,0,2.227051,2015,5,27,1,0,0,2,0.0,0,0,135,3,1,0,-0.10318,-0.091555,2,2,4.260101,0,0.0,0.0,-0.131924,2,-2.015038,-0.254873,-0.720694,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322,0.043967,-1.340370,0,2015-07-01,3,0.0,1,0
1,1,0,5.923385,2015,5,27,1,0,0,2,0.0,0,0,135,3,1,0,-0.10318,-0.091555,2,2,5.793131,0,0.0,0.0,-0.131924,2,-2.015038,-0.254873,-0.720694,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157,0.043967,-1.340370,0,2015-07-01,3,0.0,1,0
2,1,0,-0.907814,2015,5,27,1,0,1,1,0.0,0,0,59,3,1,0,-0.10318,-0.091555,0,2,-0.338990,0,0.0,0.0,-0.131924,2,-0.530935,-0.254873,-0.720694,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734,-1.340324,-0.949352,0,2015-07-01,3,0.0,0,0
3,1,0,-0.851667,2015,5,27,1,0,1,1,0.0,0,0,59,2,0,0,-0.10318,-0.091555,0,0,-0.338990,0,304.0,0.0,-0.131924,2,-0.530935,-0.254873,-0.720694,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677,-1.340324,-0.949352,0,2015-07-01,3,0.0,0,0
4,1,0,-0.842309,2015,5,27,1,0,2,2,0.0,0,0,59,6,3,0,-0.10318,-0.091555,0,0,-0.338990,0,240.0,0.0,-0.131924,2,-0.075810,-0.254873,0.540666,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498,0.043967,-0.558334,0,2015-07-01,3,0.5,0,0


In [4]:
#@title Prepare Data for Modeling

feature_names = []
target_col='is_canceled'

"""Prepare features and target variable"""
print("Preparing data for modeling...")

# Remove columns that won't be used as features

exclude_cols = ['reservation_status', 'arrival_date', 'reservation_status_date', 'name', 'email', 'phone-number', 'credit_card']
if 'reservation_status' in hotel_df.columns:
    exclude_cols.append('reservation_status')
if 'arrival_date' in hotel_df.columns:
    exclude_cols.append('arrival_date')
if 'reservation_status_date' in hotel_df.columns:
    exclude_cols.append('reservation_status_date')

features = [col for col in hotel_df.columns if col not in exclude_cols + [target_col]]
feature_names = features

X = hotel_df[features]
y = hotel_df[target_col]

print(f"Features: {len(features)}")
print(f"Samples: {X.shape[0]}")
print(f"Cancellation rate: {y.mean():.2%}")


# Prepare data
# data_preparer = DataPreparer()
# X, y = data_preparer.prepare_data(df_processed)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Data Split Summary:")
print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X):.1%})")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X):.1%})")
print(f"Number of features: {X_train.shape[1]}")

Preparing data for modeling...
Features: 36
Samples: 119390
Cancellation rate: 37.04%
Data Split Summary:
Training set: 95512 samples (80.0%)
Test set: 23878 samples (20.0%)
Number of features: 36


In [5]:
#@title ## Model Training and Comparison

feature_importance = {}
best_model = None
best_model_name = None


"""Train multiple models and select the best one"""
print("Training Multiple Models...")
print("=" * 50)

models = {
    'logistic_regression': LogisticRegression(random_state=42, max_iter=1000),
    'random_forest': RandomForestClassifier(random_state=42, n_estimators=100, n_jobs=-1),
    'gradient_boosting': GradientBoostingClassifier(random_state=42),
    'xgboost': xgb.XGBClassifier(random_state=42, eval_metric='logloss', n_jobs=-1)
}

# Parameter grids for hyperparameter tuning
param_grids = {
    'random_forest': {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5]
    },
    'xgboost': {
        'n_estimators': [100, 200],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.2]
    }
}

best_score = 0
model_results = []

for name, model in models.items():
    print(f"Training {name}...")

    if name in param_grids:
        # Perform grid search for models with parameter grids
        grid_search = GridSearchCV(model, param_grids[name],
                                    cv=5, scoring='roc_auc', n_jobs=-1, verbose=0)
        grid_search.fit(X_train, y_train)
        models[name] = grid_search.best_estimator_
        score = grid_search.best_score_
        print(f"Best parameters: {grid_search.best_params_}")
    else:
        # Use cross-validation for other models
        model.fit(X_train, y_train)
        models[name] = model
        scores = cross_val_score(model, X_train, y_train,
                                cv=5, scoring='roc_auc')
        score = np.mean(scores)

    # Store feature importance for tree-based models
    if hasattr(models[name], 'feature_importances_'):
        feature_importance[name] = models[name].feature_importances_

    model_results.append({
        'model': name,
        'roc_auc': score,
        'model_obj': models[name]
    })

    print(f"{name} - ROC-AUC: {score:.4f}")

    if score > best_score:
        best_score = score
        best_model = models[name]
        best_model_name = name

# Display results
results_df = pd.DataFrame(model_results).sort_values('roc_auc', ascending=False)
print(f"Best model: {best_model_name} with ROC-AUC: {best_score:.4f}")

# Display model comparison
print("Model Comparison:")
print(results_df[['model', 'roc_auc']].to_string(index=False))

Training Multiple Models...
Training logistic_regression...
logistic_regression - ROC-AUC: 0.8565
Training random_forest...
Best parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
random_forest - ROC-AUC: 0.9575
Training gradient_boosting...
gradient_boosting - ROC-AUC: 0.9255
Training xgboost...
Best parameters: {'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 200}
xgboost - ROC-AUC: 0.9592
Best model: xgboost with ROC-AUC: 0.9592
Model Comparison:
              model  roc_auc
            xgboost 0.959185
      random_forest 0.957479
  gradient_boosting 0.925504
logistic_regression 0.856469


In [6]:
def predict_proba(X, model_name=None):
    """Predict cancellation probabilities"""
    if model_name:
        model = models[model_name]
    else:
        model = best_model

    return model.predict_proba(X)[:, 1]

# # Make predictions with best model
y_pred_proba = predict_proba(X_test)
y_pred = (y_pred_proba >= 0.5).astype(int)

In [7]:
#@title ## Perform comprehensive model evaluation
"""Perform comprehensive model evaluation"""
# Calculate metrics
results = {}

# Basic metrics
results['accuracy'] = accuracy_score(y_test, y_pred)
results['precision'] = precision_score(y_test, y_pred)
results['recall'] = recall_score(y_test, y_pred)
results['f1_score'] = f1_score(y_test, y_pred)

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)
results['roc_auc'] = roc_auc

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall, precision)
results['pr_auc'] = pr_auc

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
results['confusion_matrix'] = cm

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
results['classification_report'] = cr

# Print evaluation metrics
print("Model Performance Metrics:")
print(f"   ROC-AUC Score: {results['roc_auc']:.4f}")
print(f"   PR-AUC Score: {results['pr_auc']:.4f}")
print(f"   Accuracy: {results['accuracy']:.4f}")
print(f"   Precision: {results['precision']:.4f}")
print(f"   Recall: {results['recall']:.4f}")
print(f"   F1-Score: {results['f1_score']:.4f}")


Model Performance Metrics:
   ROC-AUC Score: 0.9611
   PR-AUC Score: 0.9426
   Accuracy: 0.8948
   Precision: 0.8718
   Recall: 0.8393
   F1-Score: 0.8553


In [8]:
#@title ## Plot various evaluation metrics
"""Plot various evaluation metrics"""
print("  Generating Evaluation Plots...")

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('ROC Curve', 'Precision-Recall Curve',
                    'Feature Importance', 'Threshold Analysis'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}]]
)

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

fig.add_trace(
    go.Scatter(x=fpr, y=tpr, mode='lines',
                name=f'ROC curve (AUC = {roc_auc:.3f})',
                line=dict(color='darkorange', width=2)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=[0, 1], y=[0, 1], mode='lines',
                name='Random classifier',
                line=dict(color='navy', width=2, dash='dash')),
    row=1, col=1
)
fig.update_xaxes(title_text='False Positive Rate', row=1, col=1)
fig.update_yaxes(title_text='True Positive Rate', row=1, col=1)

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall, precision)

fig.add_trace(
    go.Scatter(x=recall, y=precision, mode='lines',
                name=f'PR curve (AUC = {pr_auc:.3f})',
                line=dict(color='blue', width=2)),
    row=1, col=2
)
fig.update_xaxes(title_text='Recall', row=1, col=2)
fig.update_yaxes(title_text='Precision', row=1, col=2)

# Feature Importance
if hasattr(models['xgboost'], 'feature_importances_'):
    feature_imp = pd.DataFrame({
        'feature': feature_names,
        'importance': models['xgboost'].feature_importances_
    }).sort_values('importance', ascending=True).tail(15)

    fig.add_trace(
        go.Bar(x=feature_imp['importance'], y=feature_imp['feature'],
                orientation='h', name='Feature Importance',
                marker=dict(color='lightseagreen')),
        row=2, col=1
    )
    fig.update_xaxes(title_text='Importance', row=2, col=1)
    fig.update_yaxes(title_text='Features', row=2, col=1)

# Threshold Analysis
thresholds = np.arange(0.1, 1.0, 0.1)
profits = []

for thresh in thresholds:
    y_pred = (y_pred_proba >= thresh).astype(int)
    cm = confusion_matrix(y_test, y_pred)
    # Simple profit calculation: TP saves revenue, FP has minimal cost
    profit = cm[1, 1] * 100 - cm[0, 1] * 10  # Example weights
    profits.append(profit)

fig.add_trace(
    go.Scatter(x=thresholds, y=profits, mode='lines+markers',
                name='Estimated Profit',
                line=dict(color='red', width=2)),
    row=2, col=2
)
fig.update_xaxes(title_text='Threshold', row=2, col=2)
fig.update_yaxes(title_text='Estimated Profit', row=2, col=2)

fig.update_layout(height=800, showlegend=True,
                    title_text="Model Evaluation Dashboard")
fig.show()

  Generating Evaluation Plots...


In [9]:
#@title ## Generate business insights from model predictions
"""Generate business insights from model predictions"""
print("Generating Business Insights...")

# Create results dataframe
results_df = X_test.copy()
results_df['actual_cancellation'] = y_test
results_df['predicted_probability'] = y_pred_proba
results_df['predicted_cancellation'] = (y_pred_proba >= 0.5).astype(int)

# High-risk bookings (top 20% probability)
high_risk_threshold = np.percentile(y_pred_proba, 80)
results_df['high_risk'] = results_df['predicted_probability'] >= high_risk_threshold

# Key insights
insights = {}

# High-risk customer profile
high_risk_df = results_df[results_df['high_risk'] == True]

insights['high_risk_characteristics'] = {
    'avg_lead_time': high_risk_df['lead_time'].mean() if 'lead_time' in high_risk_df.columns else 0,
    'avg_previous_cancellations': high_risk_df['previous_cancellations'].mean() if 'previous_cancellations' in high_risk_df.columns else 0,
    'high_risk_count': len(high_risk_df),
    'high_risk_percentage': len(high_risk_df) / len(results_df) * 100
}

# Revenue impact analysis
avg_room_rate = 100
potential_savings = len(high_risk_df) * avg_room_rate * 0.3  # 30% of at-risk bookings

insights['revenue_impact'] = {
    'high_risk_bookings_count': len(high_risk_df),
    'potential_revenue_at_risk': potential_savings,
    'estimated_savings_with_intervention': potential_savings * 0.5,  # 50% success rate
    'avg_room_rate': avg_room_rate
}

# Print key insights
print("\n" + "="*60)
print("KEY BUSINESS INSIGHTS")
print("="*60)

hr_char = insights['high_risk_characteristics']
print(f"\n High-Risk Booking Profile:")
print(f"   • High-risk bookings identified: {hr_char['high_risk_count']} ({hr_char['high_risk_percentage']:.1f}% of total)")
print(f"   • Average Lead Time: {hr_char['avg_lead_time']:.1f} days")
print(f"   • Average Previous Cancellations: {hr_char['avg_previous_cancellations']:.2f}")

rev_impact = insights['revenue_impact']
print(f"\n Revenue Impact Analysis:")
print(f"   • High-risk bookings: {rev_impact['high_risk_bookings_count']}")
print(f"   • Average room rate: ${rev_impact['avg_room_rate']:.2f}")
print(f"   • Potential revenue at risk: ${rev_impact['potential_revenue_at_risk']:,.2f}")
print(f"   • Estimated savings with intervention: ${rev_impact['estimated_savings_with_intervention']:,.2f}")

print(f"\n Model Performance:")
print(f"   • ROC-AUC Score: {results['roc_auc']:.4f}")
print(f"   • Precision-Recall AUC: {results['pr_auc']:.4f}")


Generating Business Insights...

KEY BUSINESS INSIGHTS

 High-Risk Booking Profile:
   • High-risk bookings identified: 4776 (20.0% of total)
   • Average Lead Time: 0.8 days
   • Average Previous Cancellations: 0.38

 Revenue Impact Analysis:
   • High-risk bookings: 4776
   • Average room rate: $100.00
   • Potential revenue at risk: $143,280.00
   • Estimated savings with intervention: $71,640.00

 Model Performance:
   • ROC-AUC Score: 0.9611
   • Precision-Recall AUC: 0.9426


In [11]:
#@title ## Actionable Recommendations

data_path = "/content/drive/MyDrive/Hotel_Cancellation_Prediction/Dataset/hotel_booking.csv"
hotel_df = pd.read_csv(data_path)
encoders = {}

encoders['deposit_type'] = LabelEncoder()
hotel_df['deposit_type'] = hotel_df['deposit_type'].astype(str)
encoders['deposit_type'].fit(hotel_df['deposit_type'])
hotel_df['deposit_type'] = encoders['deposit_type'].transform(hotel_df['deposit_type'])

encoders['market_segment'] = LabelEncoder()
hotel_df['market_segment'] = hotel_df['market_segment'].astype(str)
encoders['market_segment'].fit(hotel_df['market_segment'])
hotel_df['market_segment'] = encoders['market_segment'].transform(hotel_df['market_segment'])

print("ACTIONABLE RECOMMENDATIONS FOR HOTEL MANAGEMENT")
print("=" * 65)

print("""
1. PROACTIVE RISK MANAGEMENT

   • Deposit Strategy: Require deposits for bookings with:
     - Lead time > 100 days
     - Previous cancellation history
     - Non-refundable deposit types

   • High-Risk Alerts: Implement real-time alerts for bookings with
     > 70% cancellation probability

2. REVENUE OPTIMIZATION

   • Overbooking Strategy: Adjust overbooking levels based on
     predicted cancellation rates

   • Dynamic Pricing: Offer discounts to high-risk customers
     30 days before arrival

3. TARGETED COMMUNICATION

   • High-Risk Customers: Send personalized reminders and
     special offers 2 weeks before arrival

   • Confirmation Campaigns: Implement double-confirmation
     process for risky bookings

4. OPERATIONAL EFFICIENCY

   • Staff Allocation: Adjust staffing based on predicted
     actual arrivals vs cancellations

   • Inventory Management: Optimize room allocation and
     maintenance scheduling
""")

# High-risk customer segmentation
if 'high_risk' in results_df.columns:
    high_risk_customers = results_df[results_df['high_risk'] == True]

    print(f"\n HIGH-RISK BOOKING SEGMENTATION ({len(high_risk_customers)} bookings)")
    print("-" * 50)

    # Analyze high-risk segment characteristics
    if 'lead_time' in high_risk_customers.columns:
        print(f"   • Average lead time: {high_risk_customers['lead_time'].mean():.1f} days")

    if 'deposit_type' in high_risk_customers.columns and 'deposit_type' in encoders:
        # Get original deposit type values
        deposit_encoder = encoders['deposit_type']
        deposit_counts = high_risk_customers['deposit_type'].value_counts().head(3)
        print(f"   • Top deposit types: {list(deposit_counts.index)}")

    if 'market_segment' in high_risk_customers.columns and 'market_segment' in encoders:
        segment_encoder = encoders['market_segment']
        segment_counts = high_risk_customers['market_segment'].value_counts().head(3)
        print(f"   • Top market segments: {list(segment_counts.index)}")

print(f"\n PREDICTED BUSINESS IMPACT")
print("-" * 30)
print(f"   With proactive management, potential savings:")
print(f"   ${insights['revenue_impact']['estimated_savings_with_intervention']:,.2f}")

ACTIONABLE RECOMMENDATIONS FOR HOTEL MANAGEMENT

1. PROACTIVE RISK MANAGEMENT

   • Deposit Strategy: Require deposits for bookings with:
     - Lead time > 100 days
     - Previous cancellation history
     - Non-refundable deposit types

   • High-Risk Alerts: Implement real-time alerts for bookings with
     > 70% cancellation probability

2. REVENUE OPTIMIZATION

   • Overbooking Strategy: Adjust overbooking levels based on
     predicted cancellation rates

   • Dynamic Pricing: Offer discounts to high-risk customers
     30 days before arrival

3. TARGETED COMMUNICATION

   • High-Risk Customers: Send personalized reminders and
     special offers 2 weeks before arrival

   • Confirmation Campaigns: Implement double-confirmation
     process for risky bookings

4. OPERATIONAL EFFICIENCY

   • Staff Allocation: Adjust staffing based on predicted
     actual arrivals vs cancellations

   • Inventory Management: Optimize room allocation and
     maintenance scheduling


 HIGH-RISK B